##Import libraries

In [7]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [8]:
#Get the data (session_id, item_id, date, Datetime, Timestamp)
url01 = 'https://raw.githubusercontent.com/anhphuongnguyenquynh/session-based-recsys-fashion/main/dataset_filtered/train_session01_seq.csv'

In [9]:
url = 'https://raw.githubusercontent.com/anhphuongnguyenquynh/session-based-recsys-fashion/main/dressipi_recsys2022_datasets.zip'
!wget $url
!unzip dressipi_recsys2022_datasets.zip

--2024-06-07 09:35:44--  https://raw.githubusercontent.com/anhphuongnguyenquynh/session-based-recsys-fashion/main/dressipi_recsys2022_datasets.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79384785 (76M) [application/zip]
Saving to: ‘dressipi_recsys2022_datasets.zip’

dressipi_recsys2022 100%[===================>]  75.71M   311MB/s    in 0.2s    

2024-06-07 09:35:44 (311 MB/s) - ‘dressipi_recsys2022_datasets.zip’ saved [79384785/79384785]

Archive:  dressipi_recsys2022_datasets.zip
   creating: dressipi_recsys2022_dataset/
  inflating: dressipi_recsys2022_dataset/README.txt  
  inflating: dressipi_recsys2022_dataset/candidate_items.csv  
  inflating: dressipi_recsys2022_dataset/item_features.csv  
  inflating: dressipi_recsys2022_dataset/test_final_pu

Train/Test split

In [10]:
dataset01 = pd.read_csv(url01, index_col = 0, parse_dates=["date"])
#dataset01 = pd.read_csv('dressipi_recsys2022_dataset/train_sessions.csv')
dataset01 = dataset01.dropna()
dataset01 = dataset01.reset_index()
#fraction
#dataset = dataset01.sample(1)

In [11]:
dataset = dataset01

In [12]:
dataset.head(5)

,session_id,item_id,date,timestamp,month,weekYear,season,duration
0,18.0,4026.0,2020-08-26 19:15:47.232,1.598469e+09,8.0,35.0,2.0,753.0
1,19.0,20033.0,2020-11-02 16:42:02.287,1.604335e+09,11.0,45.0,3.0,1.0
2,19.0,6704.0,2020-11-02 16:47:36.119,1.604336e+09,11.0,45.0,3.0,1.0
3,24.0,18476.0,2020-02-26 18:24:32.770,1.582741e+09,2.0,9.0,4.0,1.0
4,36.0,26536.0,2020-06-21 10:29:51.337,1.592735e+09,6.0,25.0,2.0,1.0


In [13]:
# Filter item less than 5 interactions
df_item_count = dataset[['item_id', 'session_id']].groupby('item_id').count().sort_values(by = 'session_id', ascending = False)
df_item_count.columns = ['CountItemId']
df_item_count_5 = df_item_count[df_item_count['CountItemId'] <10]
# remove item_id less than 5 interactions
dataset = dataset[~dataset['item_id'].isin(list(df_item_count_5.index))]

In [14]:
#get dataset from latest 7 days

In [15]:
#count unique item_id
num_items = dataset['item_id'].nunique()
print('Number of items: ', num_items)

Number of items:  11954


In [16]:
from datetime import datetime, timedelta

# Get the maximum date as a string
maxdate_str = dataset['date'].max().strftime('%Y-%m-%d %H:%M:%S')

# Parse the string into a datetime object
maxdate = datetime.strptime(maxdate_str, '%Y-%m-%d %H:%M:%S')

# Calculate the date 7 days before the maximum date
mindate7 = maxdate - timedelta(days=7)

# Split the dataset into train and test sets
test_data = dataset[dataset['date'] >= mindate7]
train_data = dataset[dataset['date'] <= mindate7]

In [18]:
#Train split data test
# random_selection = np.random.rand(len(dataset.index)) <= 0.85
# train_data = dataset[random_selection]
# test_data = dataset[~random_selection]

In [19]:
# Get dictionary {SessionId:{ItemId1:Timestamp1, ItemId2:Timestamp2, ...}}
train_sess = train_data[['session_id', 'item_id', 'timestamp']].groupby('session_id').apply(lambda x: dict(zip(x['item_id'], x['timestamp'])))
test_sess = test_data[['session_id', 'item_id', 'timestamp']].groupby('session_id').apply(lambda x: dict(zip(x['item_id'], x['timestamp'])))

In [20]:
#Remove train_sess less than 2 item_id
train_sess = train_sess[train_sess.apply(lambda x: len(x) >= 2)]
test_sess = test_sess[test_sess.apply(lambda x: len(x) >= 2)]

In [21]:
train_sess

session_id
19.0         {20033.0: 1604335322.287, 6704.0: 1604335656.119}
108.0        {4816.0: 1591462767.995, 13885.0: 1591463520.6...
154.0        {21152.0: 1587713021.545, 27613.0: 1587713168....
380.0        {14208.0: 1609504400.187, 19112.0: 1609532456....
415.0        {27599.0: 1579507402.061, 4040.0: 1579507525.384}
                                   ...                        
4439895.0    {19454.0: 1605039741.558, 23687.0: 1605039891....
4439964.0    {11397.0: 1585037643.053, 10093.0: 1585037838....
4439973.0    {1648.0: 1604042533.113, 20829.0: 1604042603.2...
4439990.0    {17429.0: 1598096360.419, 22093.0: 1598099595....
4440001.0    {25129.0: 1604076172.451, 25273.0: 1604100295....
Length: 88895, dtype: object

#Preprocessing data

In [22]:
sessDict = {214834865: '1396808691.295000', 214706441: '1396808691.426000', 214820225: '1396808691.422000'}
#Function to preprocess create input and output for the sessions
def _preprocess_sess_dict(sessDict):
    sessDictTime = dict([(v, k) for (k, v) in sessDict.items()])
    sessSort = sorted(sessDictTime.items(), reverse = False)
    times = [item[0] for item in sessSort]
    itemIds = [item[1] for item in sessSort]
    inp_seq = []
    labels = []
    inp_time = []

    for i in range(len(sessSort)):
        if i >= 1:
            inp_seq += [itemIds[:i]]
            labels += [itemIds[i]]
            inp_time += [times[i]]
    return inp_seq, inp_time, labels, itemIds

inp_seq, inp_time, labels, itemIds = _preprocess_sess_dict(sessDict)
print('input sequences: ', inp_seq)
print('input times: ', inp_time)
print('targets: ', labels)
print('sequence: ', itemIds)

input sequences:  [[214834865], [214834865, 214820225]]
input times:  ['1396808691.422000', '1396808691.426000']
targets:  [214820225, 214706441]
sequence:  [214834865, 214820225, 214706441]


In [23]:
#Create input and output for sessions
def _preprocess_data(data_sess):
    inp_seqs = []
    inp_times = []
    labels = []
    sequences = []
    sessIds = list(data_sess.index)
    for sessId in sessIds:
        sessDict = data_sess.loc[sessId]
        inp_seq, inp_time, label, sequence = _preprocess_sess_dict(sessDict)
        inp_seqs += inp_seq
        inp_times += inp_time
        labels += label
        sequences += sequence
    return inp_seqs, inp_times, labels, sequences

train_inp_seqs, train_inp_dates, train_labs, train_sequences = _preprocess_data(train_sess)
test_inp_seqs, test_inp_dates, test_labs, test_sequences = _preprocess_data(test_sess)

train = (train_inp_seqs, train_labs)
test = (test_inp_seqs, test_labs)

print('Done.')

Done.


Save train/test

In [24]:
import pickle
import os

def _save_file(filename, obj):
  with open(filename, 'wb') as fn:
    pickle.dump(obj, fn)

# Create folder for save train/test
if not os.path.exists('dressipi_data_train0.2'):
  os.mkdir('dressipi_data_train0.2')

# save train/test
_save_file('dressipi_data_train0.2/train02.pkl', train)
_save_file('dressipi_data_train0.2/test02.pkl', test)

In [25]:
import pickle

def _load_file(filename):
  with open(filename, 'rb') as fn:
    data = pickle.load(fn)
  return data

# load train and test data from pickle
train = _load_file('dressipi_data_train0.2/train02.pkl')
test = _load_file('dressipi_data_train0.2/test02.pkl')

# Build vocabulary

In [26]:
# Token default
PAD_token = 0  # token padding for short sequence. If nothing, pad_token equal to 0

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1  #default number items is 1 - for PAD_token

    def addSenquence(self, data):
        for sequence in data:
          for item in sequence:
              self.addItem(item)

    # add one item into the system
    def addItem(self, item):
        if item not in self.item2index:
            self.item2index[item] = self.num_items
            self.item2count[item] = 1
            self.index2item[self.num_items] = item
            self.num_items += 1
        else:
            self.item2count[item] += 1

    #Remove item exist < min_count
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_items = []

        for k, v in self.item2count.items():
            if v >= min_count:
                keep_items.append(k)

        print('keep_items {} / {} = {:.4f}'.format(
            len(keep_items), len(self.item2index), len(keep_items) / len(self.item2index)
        ))

        # Create vocabulary again
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1

        # Add more items into vocabulary
        for item in keep_items:
            self.addItem(item)

    # convert sequence into a list of indices
    def _seqItem2seqIndex(self, x):
        return [voc.item2index[item] if item in voc.item2index else 0 for item in x]

In [27]:
#Get all the lists of items in sessionIds
from itertools import chain
seq_targets = [train[1]] + [test[1]]
sessionIds = list(chain.from_iterable(seq_targets))
sessionIds = set(sessionIds)
print('Number of sessionIds: ', len(sessionIds))

Number of sessionIds:  11903


In [28]:
#Build vocabulary
voc = Voc('DictItemId')
voc.addSenquence(seq_targets)

# Convert and test on one sequence itemIds
print('sequence of itemIds: ', train[0][7])
print('converted indices: ', voc._seqItem2seqIndex(train[0][7]))

sequence of itemIds:  [1720.0]
converted indices:  [10043]


In [29]:
#convert train, test into indices
train_x_index = [voc._seqItem2seqIndex(seq) for seq in train[0]]
test_x_index = [voc._seqItem2seqIndex(seq) for seq in test[0]]
train_y_index = voc._seqItem2seqIndex(train[1])
test_y_index = voc._seqItem2seqIndex(test[1])
train_index = (train_x_index, train_y_index)
test_index = (test_x_index, test_y_index)

#Train/Test/Validation

In [66]:
def load_data(root='', valid_portion=0.1, maxlen=13, sort_by_len=False, train_set=None, test_set=None):
    """Load dataset từ root
    root: folder dữ liệu train, trong trường hợp train_set, test_set tồn tại thì không sử dụng train_set và test_set
    valid_portion: tỷ lệ phân chia dữ liệu validation/train
    maxlen: độ dài lớn nhất của sequence
    sort_by_len: có sort theo chiều dài các session trước khi chia hay không?
    train_set: training dataset
    test_set:  test dataset
    """

    # Load the dataset
    if train_set is None and test_set is None:
        path_train_data = os.path.join(root, 'train.pkl')
        path_test_data = os.path.join(root, 'test.pkl')
        with open(path_train_data, 'rb') as f1:
            train_set = pickle.load(f1)

        with open(path_test_data, 'rb') as f2:
            test_set = pickle.load(f2)

    if maxlen:
        new_train_set_x = []
        new_train_set_y = []
        # Lọc dữ liệu sequence đến maxlen
        for x, y in zip(train_set[0], train_set[1]):
            if len(x) < maxlen:
                new_train_set_x.append(x)
                new_train_set_y.append(y)
            else:
                new_train_set_x.append(x[:maxlen])
                new_train_set_y.append(y)
        train_set = (new_train_set_x, new_train_set_y)
        del new_train_set_x, new_train_set_y

        new_test_set_x = []
        new_test_set_y = []
        for xx, yy in zip(test_set[0], test_set[1]):
            if len(xx) < maxlen:
                new_test_set_x.append(xx)
                new_test_set_y.append(yy)
            else:
                new_test_set_x.append(xx[:maxlen])
                new_test_set_y.append(yy)
        test_set = (new_test_set_x, new_test_set_y)
        del new_test_set_x, new_test_set_y

    # phân chia tập train thành train và validation
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    (test_set_x, test_set_y) = test_set

    # Trả về indices thứ tự độ dài của mỗi phần tử trong seq
    def len_argsort(seq):
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))

    # Sắp xếp session theo độ dài tăng dần
    if sort_by_len:
        sorted_index = len_argsort(test_set_x)
        test_set_x = [test_set_x[i] for i in sorted_index]
        test_set_y = [test_set_y[i] for i in sorted_index]

        sorted_index = len_argsort(valid_set_x)
        valid_set_x = [valid_set_x[i] for i in sorted_index]
        valid_set_y = [valid_set_y[i] for i in sorted_index]

    train = (train_set_x, train_set_y)
    valid = (valid_set_x, valid_set_y)
    test = (test_set_x, test_set_y)
    return train, valid, test

#Data Loader

In [31]:
from torch.utils.data import Dataset

class RecSysDataset(Dataset):
    """define the pytorch Dataset class
    """
    def __init__(self, data):
        self.data = data
        print('-'*50)
        print('Dataset info:')
        print('Number of sessions: {}'.format(len(data[0])))
        print('-'*50)

    def __getitem__(self, index):
        session_items = self.data[0][index]
        target_item = self.data[1][index]
        return session_items, target_item

    def __len__(self):
        return len(self.data[0])

In [32]:
#Hàm phụ trợ
import torch

def collate_fn(data):
    """
    Hàm số này sẽ được sử dụng để pad session về max length
    Args:
      data: batch truyền vào
    return:
      batch data đã được pad length có shape maxlen x batch_size
    """
    # Sort batch theo độ dài của input_sequence từ cao xuống thấp
    data.sort(key=lambda x: len(x[0]), reverse=True)
    lens = [len(sess) for sess, label in data]
    labels = []
    # Padding batch size
    padded_sesss = torch.zeros(len(data), max(lens)).long()
    for i, (sess, label) in enumerate(data):
        padded_sesss[i,:lens[i]] = torch.LongTensor(sess)
        labels.append(label)

    # Transpose dữ liệu từ batch_size x maxlen --> maxlen x batch_size
    padded_sesss = padded_sesss.transpose(0,1)
    return padded_sesss, torch.tensor(labels).long(), lens

#Metric

In [33]:
import torch

def get_recall(indices, targets):
    """
    Tính toán chỉ số recall cho một tập hợp predictions và targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices được dự báo từ mô hình model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
    """
    # copy targets k lần để trở thành kích thước Bxk
    targets = targets.view(-1, 1).expand_as(indices)
    # so sánh targets với indices để tìm ra vị trí mà khách hàng sẽ hit.
    hits = (targets == indices).to(device)
    hits = hits.double()
    if targets.size(0) == 0:
        return 0
    # Đếm số hit
    n_hits = torch.sum(hits)
    recall = n_hits / targets.size(0)
    return recall


def get_mrr(indices, targets):
    """
    Tính toán chỉ số MRR cho một tập hợp predictions và targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices được dự báo từ mô hình model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the MRR score
    """
    tmp = targets.view(-1, 1)
    targets = tmp.expand_as(indices)
    hits = (targets == indices).to(device)
    hits = hits.double()
    if hits.sum() == 0:
      return 0
    argsort = []
    for i in np.arange(hits.shape[0]):
      index_col = torch.where(hits[i, :] == 1)[0]+1
      if index_col.shape[0] != 0:
        argsort.append(index_col.double())
    inv_argsort = [1/item for item in argsort]
    mrr = sum(inv_argsort)/hits.shape[0]
    return mrr


def evaluate(logits, targets, k=20):
    """
    Đánh giá model sử dụng Recall@K, MRR@K scores.
    Args:
        logits (B,C): torch.LongTensor. giá trị predicted logit cho itemId tiếp theo.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
        mrr (float): the mrr score
    """
    # Tìm ra indices của topk lớn nhất các giá trị dự báo.
    _, indices = torch.topk(logits, k, -1)
    recall = get_recall(indices, targets)
    mrr = get_mrr(indices, targets)
    return recall, mrr

In [70]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

logits = torch.tensor([[1, 2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)

targets = torch.tensor([1, 2, 2]).to(device)

recall, mrr = evaluate(logits = logits, targets = targets, k = 2)
print(recall, mrr)

tensor(1., dtype=torch.float64) tensor([1.], dtype=torch.float64)


#Model NARM

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class NARM(nn.Module):
    def __init__(self, hidden_size, n_items, embedding_dim, n_layers=1, dropout=0.25):
        super(NARM, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(self.n_items, self.embedding_dim, padding_idx = 0)
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        # set bidirectional = True for bidirectional
        self.gru = nn.GRU(input_size = hidden_size, # number of expected feature of input x
                          hidden_size = hidden_size, # number of expected feature of hidden state
                          num_layers = n_layers, # number of GRU layers
                          dropout=(0 if n_layers == 1 else dropout), # dropout probability apply in encoder network
                          bidirectional=True # one or two directions.
                         )
        self.emb_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.embedding_dim, self.hidden_size, self.n_layers)
        self.a_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.a_2 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.v_t = nn.Linear(self.hidden_size, 1, bias=False)
        self.ct_dropout = nn.Dropout(0.5)
        self.b = nn.Linear(self.embedding_dim, 2 * self.hidden_size, bias=False)
        self.sf = nn.Softmax()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, input_seq, input_lengths, hidden=None):
        """
        input_seq: Batch input_sequence. Shape: max_len x batch_size
        input_lengths: Batch input lengths. Shape: batch_size
        """
        # Step 1: Convert sequence indexes to embeddings
        # shape: (max_length , batch_size , hidden_size)
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module. Padding zero when length less than max_length of input_lengths.
        # shape: (max_length , batch_size , hidden_size)
        packed = pack_padded_sequence(embedded, input_lengths)

        # Step 2: Forward packed through GRU
        # outputs is output of final GRU layer
        # hidden is concatenate of all hidden states corresponding with each time step.
        # outputs shape: (max_length , batch_size , hidden_size x num_directions)
        # hidden shape: (n_layers x num_directions , batch_size , hidden_size)
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding. Revert of pack_padded_sequence
        # outputs shape: (max_length , batch_size , hidden_size x num_directions)
        outputs, length = pad_packed_sequence(outputs)

        # Step 3: Global Encoder & Local Encoder
        # num_directions = 1 -->
        # outputs shape:(max_length , batch_size , hidden_size)
        # hidden shape: (n_layers , batch_size , hidden_size)
        # lấy hidden state tại time step cuối cùng
        ht = hidden[-1]
        # reshape outputs
        outputs = outputs.permute(1, 0, 2) # [batch_size, max_length, hidden_size] #permute to change size of tensor
        c_global = ht
        # Flatten outputs thành shape: [batch_size, max_length, hidden_size]
        gru_output_flatten = outputs.contiguous().view(-1, self.hidden_size)
        # Thực hiện một phép chiếu linear projection để tạo các latent variable có shape [batch_size, max_length, hidden_size]
        q1 = self.a_1(gru_output_flatten).view(outputs.size())
        # Thực hiện một phép chiếu linear projection để tạo các latent variable có shape [batch_size, max_length, hidden_size]
        q2 = self.a_2(ht)
        # Ma trận mask đánh dấu vị trí khác 0 trên padding sequence.
        mask = torch.where(input_seq.permute(1, 0) > 0, torch.tensor([1.], device = self.device), torch.tensor([0.], device = self.device)) # batch_size x max_len
        # Điều chỉnh shape
        q2_expand = q2.unsqueeze(1).expand_as(q1) # shape [batch_size, max_len, hidden_size]
        q2_masked = mask.unsqueeze(2).expand_as(q1) * q2_expand # batch_size x max_len x hidden_size
        # Tính trọng số alpha đo lường similarity giữa các hidden state
        alpha = self.v_t(torch.sigmoid(q1 + q2_masked).view(-1, self.hidden_size)).view(mask.size()) # batch_size x max_len
        alpha_exp = alpha.unsqueeze(2).expand_as(outputs) # batch_size x max_len x hidden_size
        # Tính linear combinition của các hidden state
        c_local = torch.sum(alpha_exp * outputs, 1) # (batch_size x hidden_size)

        # Véc tơ combinition tổng hợp
        c_t = torch.cat([c_local, c_global], 1) # batch_size x (2*hidden_size)
        c_t = self.ct_dropout(c_t)
        # Tính scores

        # Step 4: Decoder
        # embedding cho toàn bộ các item
        item_indices = torch.arange(self.n_items).to(device) # 1 x n_items
        item_embs = self.embedding(item_indices) # n_items x embedding_dim
        # reduce dimension by bi-linear projection
        B = self.b(item_embs).permute(1, 0) # (2*hidden_size) x n_items
        scores = torch.matmul(c_t, B) # batch_size x n_items
        # scores = self.sf(scores)
        return scores

In [41]:
#Kiểm tra model NARM
# Thử nghiệm model bằng cách giả lập 1 input và thực hiện quá trình feed forward
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_size = 3
n_layers = 7
# embedding = nn.Embedding(11000, hidden_size)
input_variable = torch.tensor([[  66,  369,   66, 1272],
                                [ 567,  183,   28,  616],
                                [ 392, 1558, 1143,  175],
                                [ 394,   31,   31, 5558],
                                [   0,    0,    0,    0]]).to(device)

lengths =  torch.tensor([5, 5, 5, 5]).to(device)
print('input_seq: \n', input_variable)
print('input_lengths: \n', lengths)
model_test = NARM(hidden_size = hidden_size, n_items  = 100000, embedding_dim = 100, n_layers=1, dropout=0.25).to(device)
print('model phrase: \n', model_test)
scores = model_test.forward(input_seq = input_variable, input_lengths = lengths)
print('probability distribution: ', scores.shape)

input_seq: 
 tensor([[  66,  369,   66, 1272],
        [ 567,  183,   28,  616],
        [ 392, 1558, 1143,  175],
        [ 394,   31,   31, 5558],
        [   0,    0,    0,    0]])
input_lengths: 
 tensor([5, 5, 5, 5])
model phrase: 
 NARM(
  (embedding): Embedding(100000, 100, padding_idx=0)
  (gru): GRU(100, 3)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (a_1): Linear(in_features=3, out_features=3, bias=False)
  (a_2): Linear(in_features=3, out_features=3, bias=False)
  (v_t): Linear(in_features=3, out_features=1, bias=False)
  (ct_dropout): Dropout(p=0.5, inplace=False)
  (b): Linear(in_features=100, out_features=6, bias=False)
  (sf): Softmax(dim=None)
)
probability distribution:  torch.Size([4, 100000])


In [43]:
a = torch.tensor([[1, 2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)

In [44]:
scores = model_test.forward(input_seq = input_variable, input_lengths = lengths)

In [45]:
print(scores)
print('probability distribution: ', scores.shape)

tensor([[ 0.0000, -0.2101,  0.1084,  ...,  0.0815, -0.1765, -0.2213],
        [ 0.0000, -0.4658,  0.0320,  ...,  0.0689, -0.2474, -0.5909],
        [ 0.0000, -0.2487, -0.1121,  ..., -0.2526, -1.2101, -0.7120],
        [ 0.0000,  0.8535, -0.0691,  ..., -0.2591,  0.1968,  0.9229]],
       grad_fn=<MmBackward0>)
probability distribution:  torch.Size([4, 100000])


In [46]:
values, indices = torch.topk(scores, 20, -1)

In [49]:
evaluate(logits = indices, targets = targets)

(tensor(0.7500, dtype=torch.float64), tensor([0.3107], dtype=torch.float64))

#Validation

In [72]:
def validate(valid_loader, model):
    model.eval()
    recalls = []
    mrrs = []
    with torch.no_grad():
        for seq, target, lens in valid_loader:
            seq = seq.to(device)
            target = target.to(device)
            outputs = model(seq, lens)
            logits = F.softmax(outputs, dim = 1)
            recall, mrr = evaluate(logits, target, k = args['topk'])
            recalls.append(recall)
            mrrs.append(mrr)
    recall_stack = torch.stack(recalls)
    mean_recall = torch.mean(torch.stack(recalls))
    # mean_mrr = torch.mean(torch.stack(mrrs))
    return mean_recall #, mean_mrr

#Training model

In [73]:
import os
import time
import random
import argparse
import pickle
import numpy as np
from tqdm import tqdm
from os.path import join

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn

args = {
    'dataset_path':'../input/dressipi_recsys2022_dataset/train_sessions.csv',
    'batch_size': 512,
    'hidden_size': 100,
    'embed_dim': 50,
    'epoch': 10,
    'lr':0.01,
    'lr_dc':0.1,
    'lr_dc_step':80,
    'test':None,
    'topk':20,
    'valid_portion':0.1,
    'n_items': num_items
}

here = os.path.dirname(os.getcwd())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def main():
    print('Loading data...')
    train_data, valid_data, test_data = load_data(train_set=train_index, test_set=test_index)
    train_data = RecSysDataset(train_data)
    valid_data = RecSysDataset(valid_data)
    test_data = RecSysDataset(test_data)
    train_loader = DataLoader(train_data, batch_size = args['batch_size'], shuffle = True, collate_fn = collate_fn)
    valid_loader = DataLoader(valid_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)
    test_loader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)
    print('Complete load data!')
    n_items = voc.num_items
    model = NARM(hidden_size = args['hidden_size'], n_items = args['n_items'], embedding_dim = args['embed_dim'], n_layers=1, dropout=0.25).to(device)
    print('complete load model!')

    if args['test'] == 'store_true':
        ckpt = torch.load('latest_checkpoint.pth.tar')
        model.load_state_dict(ckpt['state_dict'])
        recall = validate(test_loader, model)*10
        print("Test: Recall@{}: {:.4f}".format(args['topk'], recall, args['topk']))
        return model

    optimizer = optim.Adam(model.parameters(), args['lr'])
    criterion = nn.CrossEntropyLoss(label_smoothing = 0.1)
    scheduler = StepLR(optimizer, step_size = args['lr_dc_step'], gamma = args['lr_dc'])

    print('start training!')
    for epoch in tqdm(range(args['epoch'])):
        # train for one epoch
        trainForEpoch(train_loader, model, optimizer, epoch, args['epoch'], criterion, log_aggr = 100)
        scheduler.step(epoch = epoch)
        recall = validate(valid_loader, model)*10
        print('Epoch {} validation: Recall@{}: {:.4f}\n'.format(epoch, args['topk'], recall, args['topk']))

        # store best loss and save a model checkpoint
        ckpt_dict = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(ckpt_dict, 'latest_checkpoint.pth.tar')
    return model


def trainForEpoch(train_loader, model, optimizer, epoch, num_epochs, criterion, log_aggr=100):
    model.train()

    sum_epoch_loss = 0

    start = time.time()
    for i, (seq, target, lens) in enumerate(train_loader):
        seq = seq.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(seq, lens)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        loss_val = loss.item()
        sum_epoch_loss += loss_val

        iter_num = epoch * len(train_loader) + i + 1

        if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f) (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),
                  len(seq) / (time.time() - start)))

        start = time.time()

model = main()

Loading data...
--------------------------------------------------
Dataset info:
Number of sessions: 120219
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 13358
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 4756
--------------------------------------------------
Complete load data!
complete load model!
start training!


  0%|          | 0/10 [00:00<?, ?it/s]

[TRAIN] epoch 1/10  observation 0/235 batch loss: 10.8305 (avg 10.8305) (16.18 im/s)
[TRAIN] epoch 1/10  observation 100/235 batch loss: 9.4049 (avg 9.5171) (23.10 im/s)
[TRAIN] epoch 1/10  observation 200/235 batch loss: 9.3059 (avg 9.4503) (18.03 im/s)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
 10%|█         | 1/10 [01:14<11:12, 74.68s/it]

Epoch 0 validation: Recall@20: 0.2310

[TRAIN] epoch 2/10  observation 0/235 batch loss: 9.2242 (avg 9.2242) (15.88 im/s)
[TRAIN] epoch 2/10  observation 100/235 batch loss: 9.1462 (avg 9.1971) (28.85 im/s)
[TRAIN] epoch 2/10  observation 200/235 batch loss: 9.1577 (avg 9.1789) (19.12 im/s)


...

Epoch 7 validation: Recall@20: 0.5327

[TRAIN] epoch 9/10  observation 0/235 batch loss: 7.9322 (avg 7.9322) (32.21 im/s)
[TRAIN] epoch 9/10  observation 100/235 batch loss: 8.0572 (avg 7.9823) (33.51 im/s)
[TRAIN] epoch 9/10  observation 200/235 batch loss: 8.1434 (avg 8.0339) (18.10 im/s)


 90%|█████████ | 9/10 [10:54<01:12, 72.26s/it]

Epoch 8 validation: Recall@20: 0.5812

[TRAIN] epoch 10/10  observation 0/235 batch loss: 7.7989 (avg 7.7989) (21.39 im/s)
[TRAIN] epoch 10/10  observation 100/235 batch loss: 7.9192 (avg 7.8335) (22.45 im/s)
[TRAIN] epoch 10/10  observation 200/235 batch loss: 7.9925 (avg 7.8905) (24.23 im/s)


100%|██████████| 10/10 [12:07<00:00, 72.74s/it]

Epoch 9 validation: Recall@20: 0.5913

